In [1]:
import pandas as pd
import numpy as np

In [13]:
import os
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset # 可以繼承 類似map and unmap
import numpy as np
import pandas as pd
from collections import defaultdict
import re
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import BertTokenizer,BertModel,get_linear_schedule_with_warmup 
import seaborn as sns
# import nlpaug.augmenter.sentence as nas
# import nlpaug.augmenter.char as nac
# import nlpaug.augmenter.word as naw

ImportError: cannot import name 'is_tokenizers_available' from 'transformers.utils' (/Users/user/opt/anaconda3/lib/python3.8/site-packages/transformers/utils/__init__.py)

In [ ]:
# check the package version


In [3]:
# read the csv file
df = pd.read_csv('df_sample.csv')
df

,Unnamed: 0,meeting_text,file_name,date
0,749,"Ladies and gentlemen. Good afternoon. Welcome,...",20190424_AXA_SA-_Shareholder_Mtg_Call_2019-4-2...,2019-04-24
1,1492,Greetings. Welcome to the Markel Corporation F...,20120510_Markel_Corp-_Earnings_Call_2012-5-10_...,2012-05-10
2,446,Good morning or good afternoon. Welcome to Swi...,20190731_Swiss_Re_AG-_Earnings_Call_2019-7-31_...,2019-07-31
3,1438,"Market Cap: 69,035.17. Current PX: 148.33. YTD...",20170726_Chubb_Ltd-_Earnings_Call_2017-7-26_FS...,2017-07-26
4,337,"Yasuyoshi Karasawa. Good afternoon, ladies and...",20190524_MS-AD_INSURANCE-_Guidance_Call_2019-5...,2019-05-24
...,...,...,...,...
913,423,"Ladies and gentlemen, good morning or good aft...",20160429_Swiss_Re_AG-_Earnings_Call_2016-4-29_...,2016-04-29
914,96,"Welcome, everyone, to our presentation today. ...",20200526_Aviva_PLC-_Shareholder_Mtg_Call_2020-...,2020-05-26
915,40,"Operator. (Operator Instructions) Good day, la...",20191204_Sanlam_Ltd-_Sales_Results_Call_2019-1...,2019-12-04
916,605,"Greetings, and welcome to the Lancashire Holdi...",20150729_LANCASHIRE_HOLDI-_Earnings_Call_2015-...,2015-07-29


In [7]:
# def cleaning_text(contents):
#     ### Cleaning all the unwanted rows in the transcript
#     df = pd.DataFrame(contents)

#     # remove the unnessary string
#     df[0] = df[0].str.replace('\n','')
#     df[0] = df[0].str.replace('Bloomberg Transcript','')
#     df[0] = df[0].str.replace('\x0c\n','')
#     df[0] = df[0].str.replace('FINAL','')
#     df[0] = df[0].str.replace('A - ','')
#     df[0] = df[0].str.replace('Q - ','')

#     # using re to remove the unnessary string
#     def drop_unnessary(x):
#         page = re.findall(r'Page \d+ of \d+', x) # 'page ... of ... '
#         BIO = re.findall(r'{BIO', x) # '{BIO 18731996 <GO>}'
#         Company_Name = re.findall(r'Company N ame:', x) # 'Company N ame: H annover Rueck SE'
#         Company_Ticker = re.findall(r'Company Ticker:', x) # 'Company Ticker: H N R1 GR Equity'
#         Date = re.findall(r'Date:', x) # Date: 2015-03-10
#         if page == [] and BIO == [] and Company_Name == [] and Company_Ticker == [] and Date == []:
#             return True
#         else:
#             return False

#     true_false = df[0].apply(lambda x: drop_unnessary(x))
#     df = df[true_false]

#     # drop the final page declaration
#     df = df[df[0] != 'This transcript may not be 100 percent accurate and may contain misspellings and other']
#     df = df[df[0] != 'inaccuracies. This transcript is provided "as is", without express or implied warranties of']
#     df = df[df[0] != 'any kind. Bloomberg retains all rights to this transcript and provides it solely for your']
#     df = df[df[0] != 'personal, non-commercial use. Bloomberg, its suppliers and third-party agents shall']
#     df = df[df[0] != 'have no liability for errors in this transcript or for lost profits, losses, or direct, indirect,']
#     df = df[df[0] != 'incidental, consequential, special or punitive damages in connection with the']
#     df = df[df[0] != 'furnishing, performance or use of such transcript. Neither the information nor any']
#     df = df[df[0] != 'opinion expressed in this transcript constitutes a solicitation of the purchase or sale of']
#     df = df[df[0] != 'securities or commodities. Any opinion expressed in the transcript does not necessarily']
#     # df = df[df[0] != 'reflect the views of Bloomberg LP. ¬© COPYRIGHT 2022, BLOOMBERG LP. All rights']  
#     df = df[df[0] != 'reserved. Any reproduction, redistribution or retransmission is expressly prohibited.']
#     # ¬© could not be identified, would apply re
#     def drop_Bloomberg_mark(x):
#         Bloomberg_mark = re.findall(r'reflect the views of Bloomberg LP', x) # 'reflect the views of Bloomberg LP. ¬© COPYRIGHT 2022, BLOOMBERG LP. All rights'
#         if Bloomberg_mark == []:
#             return True
#         else:
#             return False

#     true_false = df[0].apply(lambda x: drop_Bloomberg_mark(x))
#     df = df[true_false]

#     # drop the empthy row
#     df = df[df[0] != '']
#     df = df[df[0] != '']

#     return df

def preprocess(text):
      pattern = re.compile(r'\s+')
  text=text.replace('\u202f',' ').replace('xa0',' ')
  text=re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
  text=re.sub(r"\\u00b4",r'',text)
  text=re.sub(pattern, ' ', text)
  text = text.replace('?', ' ? ').replace(')', ') ').strip()
  return text

  df['imdb_user_review']=df['imdb_user_review'].apply(preprocess)

In [8]:
df_clean = cleaning_text(df['meeting_text'])

KeyError: 0